### 📥 Carregamento da Base de Treino

Este trecho identifica automaticamente a raiz do projeto e carrega o arquivo df_trein.parquet da pasta data/processed.

Após verificar que o arquivo existe, o dataset é lido com pandas e são exibidos o tamanho (shape) e os tipos das colunas (dtypes), validando que a base está pronta para a modelagem.


In [1]:
import pandas as pd
from pathlib import Path

# Descobre a raiz do projeto (assume que o notebook está em notbooks/)
try:
    ROOT = Path(__file__).resolve().parents[1]
except NameError:
    # __file__ não existe em notebooks; usa o cwd como base
    ROOT = Path.cwd().resolve().parent

data_path = ROOT / 'data' / 'processed' / 'df_trein.parquet'

if not data_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {data_path}")

df = pd.read_parquet(data_path)

print("Shape do dataset:", df.shape)
print("\nTipos de dados:\n")
print(df.dtypes)

Shape do dataset: (1390, 19)

Tipos de dados:

ANO                   int64
IDADE                 int64
FASE                  int64
DEFASAGEM             int64
IAA                 float64
IEG                 float64
IDA                 float64
IAN                 float64
IPS                 float64
IPV                 float64
NOTA_MAT            float64
NOTA_POR            float64
ABANDONO              int64
IDA_MISSING           int64
IEG_MISSING           int64
IPV_MISSING           int64
IPS_MISSING           int64
NOTA_MAT_MISSING      int64
NOTA_POR_MISSING      int64
dtype: object


### 🌳 Treinamento e Avaliação com Random Forest (2022 → 2023)

Este bloco treina um RandomForestClassifier para prever ABANDONO, usando validação temporal (treino 2022 e teste 2023) e excluindo ANO das features para evitar viés.

O que é feito:

 - Sanity checks: confirma target e anos esperados.

 - Separação de features/target e split temporal (2022/2023).

 - Robustez numérica: substitui inf/-inf e NaN por 0 para evitar erros no treino.

 - Treinamento do Random Forest com:

 - muitos estimadores (n_estimators=1000) e profundidade controlada (max_depth=8) para reduzir overfitting,

 - class_weight para lidar com desbalanceamento,

 - oob_score=True para obter uma validação interna via out-of-bag.

 - Avaliação por probabilidade com ROC-AUC e PR-AUC no holdout 2023.

 - Escolha de threshold por regra de negócio: define o limiar para atingir recall mínimo de 0,60, priorizando capturar pelo menos 60% dos abandonos.

 - Exibe matriz de confusão, classification report e a taxa de alunos “acionados” (sinalizados) com esse threshold.


In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_recall_curve, confusion_matrix, classification_report
)

TARGET = "ABANDONO"

assert TARGET in df.columns
assert set(df["ANO"].unique()) == {2022, 2023}

features = [c for c in df.columns if c not in [TARGET, "ANO"]]

train = df[df["ANO"] == 2022].copy()
test  = df[df["ANO"] == 2023].copy()

X_train, y_train = train[features], train[TARGET]
X_test,  y_test  = test[features],  test[TARGET]

# robustez numérica
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test  = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

rf = RandomForestClassifier(
    n_estimators=1000,
    max_depth=8,
    min_samples_leaf=10,
    min_samples_split=20,
    max_features="sqrt",
    class_weight={0: 1, 1: 2},
    bootstrap=True,
    oob_score=True,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
print("OOB score:", rf.oob_score_)

proba = rf.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, proba))
print("PR-AUC :", average_precision_score(y_test, proba))

prec, rec, thr = precision_recall_curve(y_test, proba)

# threshold por regra de negócio: recall mínimo
target_recall = 0.60
idx = np.where(rec >= target_recall)[0]
thr_use = thr[idx[-1]-1] if len(idx) and idx[-1] > 0 else 0.5

pred = (proba >= thr_use).astype(int)

print("\nThr usado:", thr_use)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred, digits=3))

print(f"\nAcionados: {pred.sum()}/{len(pred)} = {pred.mean():.1%}")



OOB score: 0.6776119402985075
ROC-AUC: 0.6732122860342346
PR-AUC : 0.45208506752662714

Thr usado: 0.34506281020758434
[[345 187]
 [ 74 114]]
              precision    recall  f1-score   support

           0      0.823     0.648     0.726       532
           1      0.379     0.606     0.466       188

    accuracy                          0.637       720
   macro avg      0.601     0.627     0.596       720
weighted avg      0.707     0.637     0.658       720


Acionados: 301/720 = 41.8%


### 💾 Serialização do Modelo e Artefatos

Este bloco salva todos os artefatos necessários para produção dentro da pasta app/model.

São gerados:

 - 📦 Modelo Random Forest em dois formatos:

pickle (.pkl)

joblib (.joblib)

 - 📋 Lista de features utilizadas no treino (essencial para garantir que a API receba as variáveis na ordem correta).

 - 🎯 Threshold escolhido para classificação (usado na lógica de decisão da API).

Essa etapa garante que o modelo possa ser carregado posteriormente pela API sem necessidade de novo treinamento, seguindo boas práticas de MLOps.


In [4]:
import os
import pickle
import joblib

# ==============================
# Criar pasta app/model se não existir
# ==============================
model_dir = "../app/model"
os.makedirs(model_dir, exist_ok=True)

# ==============================
# 1️⃣ Salvar modelo com pickle
# ==============================
pkl_path = os.path.join(model_dir, "random_forest_abandono.pkl")

with open(pkl_path, "wb") as f:
    pickle.dump(rf, f)

print(f"Modelo salvo em: {pkl_path}")

# ==============================
# 2️⃣ Salvar modelo com joblib
# ==============================
joblib_path = os.path.join(model_dir, "random_forest_abandono.joblib")

joblib.dump(rf, joblib_path)

print(f"Modelo salvo em: {joblib_path}")

# ==============================
# 3️⃣ Salvar lista de features
# (ESSENCIAL para API)
# ==============================
features_path = os.path.join(model_dir, "features.pkl")

with open(features_path, "wb") as f:
    pickle.dump(features, f)

print(f"Features salvas em: {features_path}")

# ==============================
# 4️⃣ Salvar threshold utilizado
# ==============================
threshold_path = os.path.join(model_dir, "threshold.pkl")

with open(threshold_path, "wb") as f:
    pickle.dump(thr_use, f)

print(f"Threshold salvo em: {threshold_path}")

print("\n✔ Artefatos do modelo gerados com sucesso!")

Modelo salvo em: ../app/model/random_forest_abandono.pkl
Modelo salvo em: ../app/model/random_forest_abandono.joblib
Features salvas em: ../app/model/features.pkl
Threshold salvo em: ../app/model/threshold.pkl

✔ Artefatos do modelo gerados com sucesso!


### ✅ Validação do Modelo Serializado

Este trecho garante que o modelo salvo em disco pode ser recarregado e usado em produção sem alterar resultados.

O fluxo é:

 - Define os caminhos do projeto e da pasta app/model.

 - Recarrega os artefatos: modelo (joblib), lista de features e threshold.

 - Reconstrói X_test usando exatamente as mesmas features e aplica a mesma limpeza numérica (inf/NaN → 0).

 - Recalcula ROC-AUC, PR-AUC e a taxa de “acionados” usando o threshold salvo.

 - Faz um teste crítico comparando as probabilidades do modelo original (rf) com as do modelo carregado (loaded_model), medindo a diferença máxima.

Se a diferença for praticamente zero, confirma que a serialização está consistente e pronta para ser usada na API.


In [6]:
from pathlib import Path
import pickle
import joblib
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score

# ==============================
# 1️⃣ Definir caminho raiz
# ==============================
BASE_DIR = Path().resolve().parent
MODEL_DIR = BASE_DIR / "app" / "model"

# ==============================
# 2️⃣ Carregar artefatos
# ==============================
loaded_model = joblib.load(MODEL_DIR / "random_forest_abandono.joblib")

with open(MODEL_DIR / "features.pkl", "rb") as f:
    loaded_features = pickle.load(f)

with open(MODEL_DIR / "threshold.pkl", "rb") as f:
    loaded_threshold = pickle.load(f)

print("✔ Artefatos carregados com sucesso")

# ==============================
# 3️⃣ Recriar dataset de teste
# ==============================
X_test_loaded = test[loaded_features].copy()
X_test_loaded = X_test_loaded.replace([np.inf, -np.inf], np.nan).fillna(0)

y_test_loaded = y_test.copy()

# ==============================
# 4️⃣ Gerar probabilidades
# ==============================
proba_loaded = loaded_model.predict_proba(X_test_loaded)[:, 1]

# ==============================
# 5️⃣ Aplicar threshold salvo
# ==============================
pred_loaded = (proba_loaded >= loaded_threshold).astype(int)

print(f"Threshold carregado: {loaded_threshold:.4f}")

# ==============================
# 6️⃣ Validar métricas novamente
# ==============================
print("\n--- MÉTRICAS COM MODELO SERIALIZADO ---")
print("ROC-AUC:", roc_auc_score(y_test_loaded, proba_loaded))
print("PR-AUC :", average_precision_score(y_test_loaded, proba_loaded))

print("\nAcionados:", pred_loaded.sum(), "/", len(pred_loaded),
      f"= {pred_loaded.mean():.1%}")

# ==============================
# 7️⃣ Teste crítico: modelo original vs carregado
# ==============================
proba_original = rf.predict_proba(X_test)[:, 1]

diff = np.abs(proba_original - proba_loaded).max()

print("\nDiferença máxima entre modelo original e carregado:", diff)

if diff < 1e-10:
    print("✔ Modelo serializado está 100% consistente com o original.")
else:
    print("⚠ Atenção: há diferença entre os modelos.")

✔ Artefatos carregados com sucesso
Threshold carregado: 0.3451

--- MÉTRICAS COM MODELO SERIALIZADO ---
ROC-AUC: 0.6732122860342346
PR-AUC : 0.45208506752662714

Acionados: 301 / 720 = 41.8%

Diferença máxima entre modelo original e carregado: 2.220446049250313e-16
✔ Modelo serializado está 100% consistente com o original.
